[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F08%2F08.ipynb)

# Day 8

Firstly, we need some standard packages to read the text file and a vector to keep a program.

In [1]:
with Ada.Text_IO;
with Ada.Integer_Text_IO;
with Ada.Containers.Vectors;

Define instruction kind as an enumeration type and the instruction as a record type. Then we can represent a program as a vector of instructions.

In [2]:
type Instruction_Kind is (acc, jmp, nop);

type Instruction is record
   Kind     : Instruction_Kind;
   Argument : Integer;
end record;

package Instruction_Vectors is new Ada.Containers.Vectors
  (Positive, Instruction);

Program : Instruction_Vectors.Vector;

Instantiate an Enumeration_IO package to read instruction kinds.

In [3]:
package Kind_IO is new Ada.Text_IO.Enumeration_IO (Instruction_Kind);

Now read the program from input file.

In [4]:
declare
   Input  : Ada.Text_IO.File_Type;
begin
   Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/08/input");

   while not Ada.Text_IO.End_Of_File (Input) loop
      declare
         Code : Instruction;
      begin
         Kind_IO.Get (Input, Code.Kind);
         Ada.Integer_Text_IO.Get (Input, Code.Argument);
         Program.Append (Code);
      end;
   end loop;
end;

Now let's write a virtual machine, a procedure to execute the program. To prevent execution from a hang, we will track execution in `Trace` array and stop execution if see already executed instruction. The `PC` is a "program counter", while `Accumulator` is the accumulator.

In [5]:
procedure Execute
  (Program      : in out Instruction_Vectors.Vector;
   Accumulator  : out Integer;
   Hangs        : out Boolean);
--  Execute the Program and return Accumulator if it hangs or exit.

procedure Execute
  (Program      : in out Instruction_Vectors.Vector;
   Accumulator  : out Integer;
   Hangs        : out Boolean)
is
   Trace : array (Program.First_Index .. Program.Last_Index) of Boolean :=
     (others => False);
   PC : Positive := Program.First_Index;
begin
   Accumulator := 0;
      
   while PC <= Program.Last_Index loop
      declare
         Code : Instruction renames Program (PC);
      begin
         exit when Trace (PC);
            
         Trace (PC) := True;

         case Code.Kind is
            when acc =>
               Accumulator := Accumulator + Code.Argument;
               PC := PC + 1;
            when jmp =>
               PC := PC + Code.Argument;
            when nop =>
               PC := PC + 1;
         end case;
      end;
   end loop;
      
   Hangs := PC <= Program.Last_Index;
end Execute;

Print result of part 1:

In [6]:
declare
   Result : Integer;
   Hangs  : Boolean;
begin
   Execute (Program, Result, Hangs);
   pragma Assert (Hangs);
   Ada.Integer_Text_IO.Put (Result);
end;

       1521

To solve the second part let's try to mutate each instruction in the program and check if it hangs.

In [7]:
for Code of Program loop
   declare
      Save   : constant Instruction := Code;
      Result : Integer;
      Hangs  : Boolean := True;
   begin
      case Code.Kind is
         when acc =>
            null;  --  Don't mutate acc instruction
         when jmp =>
            Code.Kind := nop;
            Execute (Program, Result, Hangs);
         when nop =>
            Code.Kind := jmp;
            Execute (Program, Result, Hangs);
      end case;

      if Hangs then
         Code := Save;
      else
         Ada.Integer_Text_IO.Put (Result);
         exit;
      end if;
   end;
end loop;

       1016

----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020)